# Aula 4: Monitoramento e Fairness em MLOps

Nesta aula, vamos abordar:
- O que é monitoramento em MLOps?
- Por que fairness importa em modelos de Machine Learning?
- Como implementar checagens simples de fairness e monitoramento em pipelines de ML.

In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Carregue modelo e vetorizador já treinados
modelo_path = '../model.joblib'
vetor_path = '../vectorizer.joblib'
dados_path = '../data/tweets_limpo.csv'

if not (os.path.exists(modelo_path) and os.path.exists(vetor_path)):
    raise FileNotFoundError('Treine e salve o modelo antes de executar esta aula!')

model = joblib.load(modelo_path)
vectorizer = joblib.load(vetor_path)
df = pd.read_csv(dados_path)

## O que é Monitoramento?

Monitorar um modelo em produção significa:
- Coletar informações sobre entradas e saídas do modelo
- Detectar possíveis quebras de padrão (ex: nova distribuição dos dados)
- Identificar quando o desempenho do modelo começa a cair

Ferramentas comuns: **Evidently**, **Prometheus**, **logs simples** em arquivos ou dashboards.

Vamos simular monitoramento básico de métricas e detectar deriva de dados!

In [ ]:
# Simule recebimento de novos dados
novos_textos = [
    'Muito ruim, não gostei do atendimento.',
    'A entrega foi sensacional!',
    'Não funcionou, me decepcionei.',
    'Recomendo para todos, nota 10!'
]
novos_df = pd.DataFrame({'text': novos_textos})

# Vetorize e faça predições
novos_vetores = vectorizer.transform(novos_df['text'])
novos_preds = model.predict(novos_vetores)
novos_df['sentimento_predito'] = novos_preds
print(novos_df)

# Monitoramento: porcentagem de cada classe
class_dist = novos_df['sentimento_predito'].value_counts(normalize=True)
print('Distribuição dos sentimentos preditos nos novos dados:')
print(class_dist)

## O que é Fairness?

Fairness (equidade) significa que o modelo não deve apresentar vieses injustos contra determinados grupos, situações ou padrões.
No contexto de textos, podemos analisar se o modelo está errando mais em frases curtas, ou se ele associa injustamente certas palavras a um sentimento.

Vamos analisar fairness por tamanho de texto!

In [ ]:
# Adicione coluna de tamanho dos textos
df['text_len'] = df['text'].apply(len)
df['len_category'] = pd.cut(df['text_len'], bins=[0,50,150,1000], labels=['curto', 'medio', 'longo'])

# Predições no conjunto de validação
vetores = vectorizer.transform(df['text'])
df['pred'] = model.predict(vetores)

# Análise de acurácia por categoria de tamanho
for cat in df['len_category'].unique():
    subset = df[df['len_category'] == cat]
    if not subset.empty:
        acuracia = (subset['label'] == subset['pred']).mean()
        print(f'Acurácia para textos {cat}: {acuracia:.2f} (N={len(subset)})')

## Desafio Fairness

- Analise o resultado das acurácias por tamanho de texto.
- O modelo está menos justo com algum grupo? Por quê?
- Sugira e teste melhorias (ex: aumentar dados de um grupo sub-representado).

Extra: Que outras formas de fairness poderiam ser analisadas neste contexto?

In [ ]:
for cat in df['len_category'].unique():
    subset = df[df['len_category'] == cat]
    if not subset.empty:
        print(f'\nMatriz de confusão para textos {cat}:')
        print(confusion_matrix(subset['label'], subset['pred']))
        sns.heatmap(confusion_matrix(subset['label'], subset['pred']),
                    annot=True, fmt='d', cmap='Blues',
                    xticklabels=['negativo', 'positivo'],
                    yticklabels=['negativo', 'positivo'])
        plt.title(f'Textos {cat}')
        plt.xlabel('Predito')
        plt.ylabel('Real')
        plt.show()